In [51]:
import random
import math
import simpy
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
# Atencion en el Banco Union
SEMILLA = 30
NUM_CAJEROS = 5
TIEMPO_ATENCION_MIN = 10
TIEMPO_ATENCION_MAX = 38
T_LLEGADAS = 20
TIEMPO_SIMULACION = 120
TOT_CLIENTES = 13
 
te  = 0.0 # tiempo de espera total
dt  = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza

In [56]:
def Atencion(cliente):
    global dt  #Accedemos a la variable dt declarada anteriormente
    R = random.random()  # numero aleatorio y lo guarda en R
    tiempo = TIEMPO_ATENCION_MAX - TIEMPO_ATENCION_MIN  
    tiempo_aten = TIEMPO_CORTE_MIN + (tiempo*R) # Distribucion uniforme
    yield env.timeout(tiempo_aten) # deja correr el tiempo n minutos
    print("3er paso: Atencion listo a %s en %.2f minutos" % (cliente,tiempo_aten))
    dt = dt + tiempo_aten # Acumula los tiempos de uso de la i


def cliente (env, nombre, personal ):
    global te
    global fin
    llega = env.now # Guarda el minuto de llegada del cliente
    print ("1ro paso: %s llego al Banco en el minuto %.2f" % (nombre, llega))
    with personal.request() as request: # Espera su turno
        yield request # Obtiene turno
        pasa = env.now # Guarda el minuto cuado comienza a ser atendido
        espera = pasa - llega # Calcula el tiempo que espero
        te = te + espera # Acumula los tiempos de espera
        print ("2do paso: %s pasa con el cajero en el minuto %.2f habiendo esperado %.2f" % (nombre, pasa, espera))
        yield env.process(Atencion(nombre)) # Invoca al proceso Atencion
        deja = env.now #Guarda el minuto en que termina el proceso Atencion 
        print ("4to paso: %s deja el Banco en el minuto %.2f" % (nombre, deja))
        fin = deja # Conserva globalmente el ultimo minuto de la simulacion

def principal (env, personal):
    llegada = 0
    i = 0
    for i in range(TOT_CLIENTES): # Para n clientes
        R = random.random()
        llegada = -T_LLEGADAS * math.log(R) # Distribucion exponencial
        yield env.timeout(llegada)  # Deja transcurrir un tiempo entre uno y otro
        i += 1
        env.process(cliente(env, 'Cliente %d' % i, personal)) 

In [57]:
print ("============= SIMULACION COLA EN EL BANCO=============")
random.seed (SEMILLA)  # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulacion
personal = simpy.Resource(env, NUM_PELUQUEROS) #Crea los recursos 
env.process(principal(env, personal)) #Invoca el proceso princial
env.run() #Inicia la simulacion

============= SIMULACION COLA EN EL BANCO=============
1ro paso: Cliente 1 llego al Banco en el minuto 12.36
2do paso: Cliente 1 pasa con el cajero en el minuto 12.36 habiendo esperado 0.00
3er paso: Atencion listo a Cliente 1 en 20.84 minutos
4to paso: Cliente 1 deja el Banco en el minuto 33.20
1ro paso: Cliente 2 llego al Banco en el minuto 37.17
2do paso: Cliente 2 pasa con el cajero en el minuto 37.17 habiendo esperado 0.00
1ro paso: Cliente 3 llego al Banco en el minuto 45.67
3er paso: Atencion listo a Cliente 2 en 25.88 minutos
4to paso: Cliente 2 deja el Banco en el minuto 63.05
2do paso: Cliente 3 pasa con el cajero en el minuto 63.05 habiendo esperado 17.38
1ro paso: Cliente 4 llego al Banco en el minuto 72.83
1ro paso: Cliente 5 llego al Banco en el minuto 81.70
1ro paso: Cliente 6 llego al Banco en el minuto 81.93
3er paso: Atencion listo a Cliente 3 en 31.12 minutos
4to paso: Cliente 3 deja el Banco en el minuto 94.17
2do paso: Cliente 4 pasa con el cajero en el minuto 94.1

In [59]:
print ("\n---------------------------------------------------------------------")
print ("\nINDICADORES: ")
lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_CLIENTES
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_CAJEROS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")
 


---------------------------------------------------------------------

INDICADORES: 

Longitud promedio de la cola: 8.45
Tiempo de espera promedio = 291.56
Uso promedio de la instalacion = 0.58

---------------------------------------------------------------------
